In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# plotly.offline.init_notebook_mode(connected=True)
import plotly.offline as pyoffline
import plotly.figure_factory as ff
pyoffline.init_notebook_mode(connected=True)

from scipy.cluster.hierarchy import linkage, dendrogram


print(__version__)

3.1.0


In [30]:
train=pd.read_csv("loan_train.csv")
train.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [40]:
train['Gender'].fillna(train['Gender'].mode()[0], inplace=True)
train['Married'].fillna(train['Married'].mode()[0], inplace=True)
train['Dependents'].fillna(train['Dependents'].mode()[0], inplace=True)
train['Self_Employed'].fillna(train['Self_Employed'].mode()[0], inplace=True)
train['Credit_History'].fillna(train['Credit_History'].mode()[0], inplace=True)
train['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0], inplace=True)

train['Dependents'].replace(('0', '1', '2', '3+'), (0, 1, 2, 3),inplace=True)

table = train.pivot_table(values='LoanAmount', index='Self_Employed' ,columns='Education', aggfunc=np.median)

def fage(x):
 return table.loc[x['Self_Employed'],x['Education']]

train['LoanAmount'].fillna(train[train['LoanAmount'].isnull()].apply(fage, axis=1), inplace=True)

In [94]:
np.percentile(train["ApplicantIncome"], 50)

3812.5

In [64]:
after = train[['Gender','Married','Dependents','ApplicantIncome','Loan_Status']]
after.head()

,Gender,Married,Dependents,ApplicantIncome,Loan_Status
0,Male,No,0,5849,Y
1,Male,Yes,1,4583,N
2,Male,Yes,0,3000,Y
3,Male,Yes,0,2583,Y
4,Male,No,0,6000,Y


In [65]:
after.isnull().sum()

Gender             0
Married            0
Dependents         0
ApplicantIncome    0
Loan_Status        0
dtype: int64

In [71]:
filter_data=after.loc[(after['Gender'] == "Male") & (after['Married']== "Yes") & (after['Dependents']>= 1) & (after['ApplicantIncome']>= 3812.5)]

In [72]:
filter_data.head()

,Gender,Married,Dependents,ApplicantIncome,Loan_Status
1,Male,Yes,1,4583,N
5,Male,Yes,2,5417,Y
8,Male,Yes,2,4006,Y
9,Male,Yes,1,12841,N
21,Male,Yes,1,5955,Y


In [74]:
filter_data["Loan_Status"].value_counts()

Y    84
N    34
Name: Loan_Status, dtype: int64

In [84]:
84/34


2.4705882352941178

In [89]:
other_data=after[~after.isin(filter_data)].dropna()

In [90]:
other_data.head()

,Gender,Married,Dependents,ApplicantIncome,Loan_Status
0,Male,No,0.0,5849.0,Y
2,Male,Yes,0.0,3000.0,Y
3,Male,Yes,0.0,2583.0,Y
4,Male,No,0.0,6000.0,Y
6,Male,Yes,0.0,2333.0,Y


In [91]:
other_data["Loan_Status"].value_counts()

Y    338
N    158
Name: Loan_Status, dtype: int64

In [92]:
338/158

2.1392405063291138

0